Part 4 of the LLM per game TLDR generation project

some of the game contains the topic stuff, some of the game does not

assume we target with games with custom topics

In [1]:
import pandas as pd
import numpy as np

from pathlib import Path
import os, sys, pickle, json, traceback
from datetime import datetime

In [2]:
game_steamid = 1716740              # starfield
game_name = 'starfield'             # also the folder name where the reviews are stored

# game_steamid = 1118010
# game_name = 'monster_hunter_world_iceborne'

# game_steamid = 582010
# game_name = 'monster_hunter_world'

# game_steamid = 2138330          # cyberpunk2077 phantom liberty
# game_name = 'cyberpunk2077_phantom_liberty'

# game_steamid = 1091500          # cyberpunk2077
# game_name = 'cyberpunk2077'

# game_steamid = 730
# game_name = 'counter-strike_2'

# game_steamid = 570
# game_name = "dota2"


In [37]:
# load the reviews from folder

reviews_reqs = []

# get existing folder and retrieve the cursor object (?)

# load the latest file
game_folder = Path(f'../../dataset/data_scraping/steam_comments_scraping/{game_name}')
if game_folder.exists():
    try:
        latest_file_path = game_folder.joinpath(f'steam_reviews_{game_steamid}_unique_with_gendata_with_analysis.pkl')
        with open(latest_file_path, 'rb') as f:
            reviews_reqs = pickle.load(f)           # retrieve the list of reviews
            print('Loaded:', latest_file_path)
    except IndexError as e:
        print('Error loading the latest file:', e)
        traceback.print_exc()

print()

# also load different generated data
ageReviews_df = pd.read_csv(game_folder.joinpath(f'steam_reviews_{game_steamid}_unique_with_gendata_with_analysis_ageGroup.csv'), index_col=None)
ageReviews_df = ageReviews_df.dropna()
print('Loaded ageReviews_df')
genderReviews_df = pd.read_csv(game_folder.joinpath(f'steam_reviews_{game_steamid}_unique_with_gendata_with_analysis_genderReviews.csv'), index_col=None)
genderReviews_df = genderReviews_df.dropna()
genderReviews_df = genderReviews_df[genderReviews_df['gender'].isin(['MALE', 'FEMALE'])]
print('Loaded genderReviews_df')
sentimentReviews_df = pd.read_csv(game_folder.joinpath(f'steam_reviews_{game_steamid}_unique_with_gendata_with_analysis_sentimentReviews.csv'), index_col=None)
sentimentReviews_df = sentimentReviews_df.dropna()
print('Loaded sentimentReviews_df')
sentimentReviews_truelabel_df = pd.read_csv(game_folder.joinpath(f'steam_reviews_{game_steamid}_unique_with_gendata_with_analysis_sentimentReviews_truelabel.csv'), index_col=None)
print('Loaded sentimentReviews_truelabel_df')
sentimentByAgeGroup_df = pd.read_csv(game_folder.joinpath(f'steam_reviews_{game_steamid}_unique_with_gendata_with_analysis_sentimentByAgeGroup.csv'), index_col=None)
print('Loaded sentimentByAgeGroup_df')
sentimentByGender_df = pd.read_csv(game_folder.joinpath(f'steam_reviews_{game_steamid}_unique_with_gendata_with_analysis_sentimentByGender.csv'), index_col=None)
print('Loaded sentimentByGender_df')


# load the topic modeling analysis
topicFreq_df = pd.read_csv(game_folder.joinpath(f'steam_reviews_{game_steamid}_unique_with_gendata_with_analysis_topic_freq.csv'), index_col=None)
print('Loaded topicFreq_df')

Loaded: ../../dataset/data_scraping/steam_comments_scraping/starfield/steam_reviews_1716740_unique_with_gendata_with_analysis.pkl

Loaded ageReviews_df
Loaded genderReviews_df
Loaded sentimentReviews_df
Loaded sentimentReviews_truelabel_df
Loaded sentimentByAgeGroup_df
Loaded sentimentByGender_df
Loaded topicFreq_df


In [27]:
ageReviews_df

,age_group,count
0,13-19,9833
1,20-29,38403
2,30-39,29110
3,40-49,14313
4,50-59,4800
5,60-69,0
6,70-79,0
7,80-89,0
8,90-99,0


In [28]:
genderReviews_df

,gender,count
0,MALE,77200
1,FEMALE,19259


In [29]:
sentimentReviews_df

,sentiment,count
0,POSITIVE,54710
1,NEGATIVE,41749


In [30]:
sentimentByAgeGroup_df

,sentiment,age_group,count
0,NEGATIVE,13-19,4203.0
1,NEGATIVE,20-29,16643.0
2,NEGATIVE,30-39,12712.0
3,NEGATIVE,40-49,6145.0
4,NEGATIVE,50-59,2046.0
5,POSITIVE,13-19,5630.0
6,POSITIVE,20-29,21760.0
7,POSITIVE,30-39,16398.0
8,POSITIVE,40-49,8168.0
9,POSITIVE,50-59,2754.0


In [5]:
sentimentByGender_df

,sentiment,gender,count
0,NEGATIVE,FEMALE,8321.0
1,NEGATIVE,MALE,33428.0
2,POSITIVE,FEMALE,10938.0
3,POSITIVE,MALE,43772.0


In [38]:
topicFreq_df

,Topic,Count,Topic Name
0,-1,21845,Disappointing experience with too much repetition
1,0,44929,Fun Space Exploration
2,1,9707,Underwhelming experience with frequent crashes.
3,2,2517,10/10
4,3,14577,Good Game
5,4,1454,Good graphics
6,5,77,Spaceship and fast travel
7,6,350,Lies and Disguise
8,7,594,Masterpiece
9,8,186,Underwhelming Experience with Limited Customiz...


In [6]:
from langchain_community.llms import Ollama
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from transformers import AutoTokenizer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader, DirectoryLoader
# from langchain.vectorstores import Chroma

/Users/michaelcheng/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
# prompt constants
# mistral_ai suggested model selection: https://docs.mistral.ai/guides/model-selection/
# prompt tutorial(?): https://python.langchain.com/docs/use_cases/csv
# prompt example: https://smith.langchain.com/public/b1309290-7212-49b7-bde2-75b39a32b49a/r/880c5250-5409-4346-8315-c30bc9638e18

SYSTEM_PROMPT_TEMPLATE = \
'''You have access to a pandas dataframe `df`. {description_of_the_table}
Here is the output of `df.to_markdown()`:
```
{df_markdown}
```
'''

# TODO: can further optimize?? Currently this is copied from IELTS academic writing task 1
PROMPT_TEMPLATE = \
'''Summarise the information by selecting and reporting the main features, and main comparisons where relevant. {df_specific_task}
Output a paragraph with less than 50 words.
Only output the paragraph. Do NOT output other text.'''


DESCRIPTION_AGEREVIEW_DF = \
'''df is a dataframe that describes the distribution of reviews by age group.'''

DESCRIPTION_GENDERREVIEW_DF = \
'''df is a dataframe that describes the distribution of reviews by gender'''

DESCRIPTION_SENTIMENT_DF = \
'''df is a dataframe that describes the distribution of reviews with different sentiments (positive and negative)'''

DESCRIPTION_SENTIMENT_BY_AGEGROUP_DF = \
'''df is a dataframe that describes the distribution of positive and negative reviews by age group'''

DESCRIPTION_SENTIMENT_BY_GENDER_DF = \
'''df is a frame that describes the distribution of positive and negative reviews by gender'''


SPECIFIC_TASK_REQ_AGEREVIEW_DF = \
'''Describe the most common age group that reviews the game.'''

SPECIFIC_TASK_REQ_GENDERREVIEW_DF = \
''''''

SPECIFIC_TASK_REQ_SENTIMENT_DF = \
'''Describe whether positive or negative reviews are more common, and how common are they.'''

SPECIFIC_TASK_REQ_SENTIMENT_BY_AGEGROUP_DF = \
'''Describe the most common sentiment across different age group.'''

SPECIFIC_TASK_REQ_SENTIMENT_BY_GENDER_DF = \
''''''
# '''Describe the common sentiment across players with different gender'''


In [8]:
llm_mistral7b = Ollama(
    model='mistral:7b-instruct-v0.2-q4_0', temperature=0.4,
    num_gpu = 1,        # disable/enable gpu for testing
)

In [9]:
# define callbacks for detecting token usage (Ollama only)
# copoed from llm_rag_comparison.ipynb

from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_core.outputs.llm_result import LLMResult
from collections import deque

class TokenUsageCallbackHandler(BaseCallbackHandler):

    def __init__(self, deque: deque = None):
        super().__init__()
        self.deque = deque

    def on_llm_end(self, response: LLMResult, **kwargs) -> None:
        # print(response)

        generation = response.generations[0][0]
        gen_info = generation.generation_info

        # get token usage
        # ref: https://github.com/orgs/langfuse/discussions/1179
        token_usage = gen_info.get('prompt_eval_count', 0) + gen_info.get('eval_count', 0)
        # get time costed (local machine)
        # instead of getting total duration, we get the prompt_eval_duration and eval_duration to exclude the load duration (e.g. to load the model to the gpu, etc.)
        time_costed = gen_info.get('prompt_eval_duration', 1e-10) + gen_info.get('eval_duration', 1e-10)     # in ns, a small value to indicate a inf time when it fails


        # create an object to store the token usage and time costed
        token_usage_obj = {
            'token_usage': token_usage,
            'time_costed': time_costed
        }

        # append the object to the deque
        self.deque.append(token_usage_obj)


def combine_token_usage_obj(token_usage_obj_list:list[dict]) -> dict:
    token_usage = 0
    time_costed = 0
    for obj in token_usage_obj_list:
        token_usage += obj['token_usage']
        time_costed += obj['time_costed']

    return {
        'token_usage': token_usage,
        'time_costed': time_costed
    }



common_deque = deque()
chain_config = {
    "callbacks": [TokenUsageCallbackHandler(common_deque)],
}

In [35]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

dfs = [
    ageReviews_df,
    genderReviews_df,
    sentimentReviews_df,
    sentimentByAgeGroup_df,
    sentimentByGender_df
]

prompts_per_dfs = [DESCRIPTION_AGEREVIEW_DF, DESCRIPTION_GENDERREVIEW_DF, DESCRIPTION_SENTIMENT_DF, DESCRIPTION_SENTIMENT_BY_AGEGROUP_DF, DESCRIPTION_SENTIMENT_BY_GENDER_DF]
specific_task_reqs = [
    SPECIFIC_TASK_REQ_AGEREVIEW_DF,
    SPECIFIC_TASK_REQ_GENDERREVIEW_DF,
    SPECIFIC_TASK_REQ_SENTIMENT_DF,
    SPECIFIC_TASK_REQ_SENTIMENT_BY_AGEGROUP_DF,
    SPECIFIC_TASK_REQ_SENTIMENT_BY_GENDER_DF
]

responses = []

for df, description_of_the_table, specific_task_req in zip(dfs, prompts_per_dfs, specific_task_reqs):

    chat_prompt_01 = ChatPromptTemplate.from_messages([
        ("system", SYSTEM_PROMPT_TEMPLATE),
        ("user", PROMPT_TEMPLATE),
    ])

    chain_01 = chat_prompt_01 | llm_mistral7b

    resp_01 = chain_01.invoke({
        'df_markdown': df.to_markdown(),
        'df_specific_task': specific_task_req,
        'description_of_the_table': description_of_the_table,
    }, config=chain_config)

    token_usage_01 = common_deque.popleft()

    responses.append(resp_01)

    print(resp_01)
    print(token_usage_01)
    

 The dataframe `df` reveals that the age groups 20-29 (38,403 reviews) and 30-39 (29,110 reviews) account for the majority of game reviews. The most common age group is therefore individuals in their twenties and thirties.
{'token_usage': 371, 'time_costed': 1837203000}
 The dataframe consists of two categories: 'gender' with two labels 'MALE' and 'FEMALE', and 'count' representing the number of reviews for each gender. The distribution shows that MALE received 77,200 reviews while FEMALE received 19,259 reviews.
{'token_usage': 196, 'time_costed': 1451317000}
 The dataframe `df` indicates that there are a total of 96,459 reviews, with a predominance of positive reviews (54,710 or approximately 57%) compared to negative ones (41,749 or roughly 43%).
{'token_usage': 217, 'time_costed': 1324589000}
 The dataframe shows that individuals aged between 20-29 have the highest number of negative reviews (16643), while those in the same age group also have the most positive reviews (21760). Ove

In [ ]:
# group the responses and provide a paragraph with the game description and the analysis

---

how about passing all dfs to a single prompt?

Result: attention shifted, and hallunciation exists (especially in numbers) -> stay with one df per prompt

In [20]:
# prompt constants
# mistral_ai suggested model selection: https://docs.mistral.ai/guides/model-selection/
# prompt tutorial(?): https://python.langchain.com/docs/use_cases/csv
# prompt example: https://smith.langchain.com/public/b1309290-7212-49b7-bde2-75b39a32b49a/r/880c5250-5409-4346-8315-c30bc9638e18

SYSTEM_PROMPT_TEMPLATE = \
'''You have access to multiple pandas dataframes known as `df_genderReviews`, `df_ageReviews`, `df_sentimentReviews`, `df_sentimentByAgeGroup` and `df_sentimentByGender`. They store the distributions of reviews towards a game grouped by either age, gender, or both.

Here is the output of `df_genderReviews.to_markdown()`:
```
{df_genderReviews}
```

Here is the output of `df_ageReviews.to_markdown()`:
```
{df_ageReviews}
```


Here is the output of `df_sentimentReviews.to_markdown()`:
```
{df_sentimentReviews}
```

Here is the output of `df_sentimentByAgeGroup.to_markdown()`:
```
{df_sentimentByAgeGroup}
```

Here is the output of `df_sentimentByGender.to_markdown()`:
```
{df_sentimentByGender}
```
'''

# TODO: can further optimize?? Currently this is copied from IELTS academic writing task 1
PROMPT_TEMPLATE = \
'''Summarise the information by selecting and reporting the main features in distribution of positive and negative reviews among different age groups and gender.
First describe the overall distribution of positive and negative reviews of all reviews by referencing df_sentimentReviews. Then describe the distribution of positive and negative reviews among different age. Finally, describe the distribution of positive and negative reviews among different gender.
The paragraph should begin with "The game has ...".
Output a paragraph with less than 150 words.
Only output the paragraph. Do NOT output other text.'''


In [21]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

chat_prompt_02 = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT_TEMPLATE),
    ("user", PROMPT_TEMPLATE),
])

chain_02 = chat_prompt_02 | llm_mistral7b

resp_02 = chain_02.invoke({
    'df_ageReviews': sentimentByAgeGroup_df.to_markdown(),
    'df_genderReviews': sentimentByGender_df.to_markdown(),
    'df_sentimentReviews': sentimentReviews_df.to_markdown(),
    'df_sentimentByAgeGroup': sentimentByAgeGroup_df.to_markdown(),
    'df_sentimentByGender': sentimentByGender_df.to_markdown(),
    # 'description_of_the_table': DESCRIPTION_AGEREVIEW_DF,
}, config=chain_config)

token_usage_02 = common_deque.popleft()

print(resp_02)
print(token_usage_02)


# experiment result: more numbers/dataframes result in more hallucination... Should break the sentences into smaller parts, then ask the model to group them to a paragraph.
    

 The game has a total of 54,710 reviews, with 21,760 (39.8%) being positive and 41,749 (78.2%) negative based on df_sentimentReviews. Diving deeper into the age distribution, df_ageReviews reveals that among the 13-19 age group, there are 5,630 positive reviews (65.1%) and 4,203 negative reviews (34.9%). For the 20-29 age group, there are 21,760 positive reviews (68.6%) and 16,643 negative reviews (51.4%). Similarly, for the 30-39 age group, there are 16,398 positive reviews (61.7%) and 12,712 negative reviews (48.3%), while for the 40-49 age group, there are 8,168 positive reviews (35.7%) and 6,145 negative reviews (26.3%), and finally, for the 50-59 age group, there are 2,754 positive reviews (25.4%) and 2,046 negative reviews (48.6%). Regarding gender distribution, df_genderReviews shows that among females, there are 10,938 positive reviews (56.1%) and 8,321 negative reviews (43.9%), while for males, there are 43,772 positive reviews (52.6%) and 33,428 negative reviews (47.4%).
{'to

---

Bonus: read the tm file?

In [39]:
topicFreq_df

,Topic,Count,Topic Name
0,-1,21845,Disappointing experience with too much repetition
1,0,44929,Fun Space Exploration
2,1,9707,Underwhelming experience with frequent crashes.
3,2,2517,10/10
4,3,14577,Good Game
5,4,1454,Good graphics
6,5,77,Spaceship and fast travel
7,6,350,Lies and Disguise
8,7,594,Masterpiece
9,8,186,Underwhelming Experience with Limited Customiz...


In [44]:
d = topicFreq_df.sort_values(by='Count', ascending=False).head(5)

In [45]:
d['Topic Name'].to_list()

['Fun Space Exploration',
 'Disappointing experience with too much repetition',
 'Good Game',
 'Underwhelming experience with frequent crashes.',
 '10/10']

---

The last step is in llm_main.py